# Boilerplate

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio
import time

from nwv2_python_wrapper import *
import nwv2_python_wrapper

# Run

List all Python wrappers

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['BroadcastChatMessageW',
 'EndpointW',
 'FilterCmdW',
 'FilterInterface',
 'FilterModeW',
 'FilterNoticeW',
 'FilterRspW',
 'GameOptionsW',
 'GameOutcomeW',
 'GameUpdateW',
 'GenPartInfoW',
 'GenStateDiffPartW',
 'GenStateDiffW',
 'NetRegionW',
 'PacketSettingsW',
 'PacketW',
 'PlayerInfoW',
 'ProcessUniqueIdW',
 'RequestActionW',
 'ResponseCodeW',
 'RoomListW',
 'TransportCmdW',
 'TransportInterface',
 'TransportNoticeW',
 'TransportRspW',
 'UniUpdateW',
 'debug_hello',
 'init_logging',
 'new_transport_interface']

## Transport layer setup

Create the Transport layer and run it in the background.

In [3]:
tiface_inner = await new_transport_interface("0.0.0.0", 0)
tiface_inner.run()

<Future pending cb=[<builtins.PyDoneCallback object at 0x7f47f19d6b90>()]>

## Wrap the PYO3 Transport Interface in a Python class with the same methods

Note that there is no `run()`. Hence, the passed in `TransportInterface` instance must have already had its `run()` method called.

In [4]:
class LoggingTransportInterface:
    def __init__(self, inner):
        self.inner = inner
        self.ctr = 0
        self.skipped_count = None
        
    async def command_response(self, transport_cmd):
        print(f"LTI{self.ctr}{self.skipped()}: received transport_cmd {transport_cmd}")
        self.ctr += 1
        transport_rsp = await self.inner.command_response(transport_cmd)
        print(f"LTI{self.ctr}: got transport_rsp back from Transport layer: {transport_rsp}")
        self.ctr += 1
        return transport_rsp
    
    def get_notifications(self):
        transport_notif_list = self.inner.get_notifications()
        if self.skipped_count is None:
            print(f"LTI{self.ctr}: got transport notifications: {transport_notif_list}")
            self.skipped_count = 0
            if len(transport_notif_list) > 0:
                self.reset()
        else:
            self.skipped_count += 1
        self.ctr += 1
        return transport_notif_list
    
    def skipped(self):
        if self.skipped_count is None:
            return ""
        count = self.skipped_count
        self.skipped_count = None
        return f" skipped {count}"
    
    def reset(self):
        self.skipped_count = None
        
tiface = LoggingTransportInterface(tiface_inner)
tiface

# Filter layer setup

In [5]:
fiface = FilterInterface(tiface, FilterModeW("client"))

Find methods to run

In [6]:
[m for m in dir(fiface) if not m.startswith('__')]

['command',
 'command_response',
 'get_notifications',
 'notif_poll_ms',
 'response',
 'run']

Run Filter!

In [7]:
run_fut = fiface.run()

# After waiting a bit, the above future should not have completed
time.sleep(0.1)
assert not run_fut.done()

LTI0: got transport notifications: []


Get the number of milliseconds between calling `get_notifications` on the contained `TransportInterface`. This can also be assigned to at any time. Lower values can cause performance issues, while higher values add latency for anything involving filter notifications.

In [8]:
fiface.notif_poll_ms

30

In [9]:
fiface.notif_poll_ms = 100

## Filter layer operation

### As a client, connect to server

In [10]:
import random
id = random.randint(1,10000)
connect_action = RequestActionW("connect", name="FilterNotebookClient{}".format(id), client_version="0.0.666")
connect_action

Connect { name: "FilterNotebookClient8580", client_version: "0.0.666" }

In [11]:
chococat_endpoint = EndpointW("157.230.134.224:2016")
filter_cmd = FilterCmdW("sendrequestaction", endpoint=chococat_endpoint, action=connect_action)
filter_cmd

SendRequestAction { endpoint: Endpoint(157.230.134.224:2016), action: Connect { name: "FilterNotebookClient8580", client_version: "0.0.666" } }

In [12]:
await fiface.command_response(filter_cmd)

LTI80 skipped 79: received transport_cmd NewEndpoint { endpoint: Endpoint(157.230.134.224:2016), timeout: 5s }
LTI81: got transport_rsp back from Transport layer: TransportRsp::Accepted
LTI82: received transport_cmd SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 0 }, retry_interval: 250ms }], packets: [Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient8580", client_version: "0.0.666" } }] }


Accepted

LTI83: got transport_rsp back from Transport layer: TransportRsp::Accepted
LTI84: got transport notifications: []


In [13]:
fiface.get_notifications()

[NewResponseCode { endpoint: Endpoint(157.230.134.224:2016), code: LoggedIn { cookie: "Q-xdaM5GMaRq-z7S", server_version: "0.3.5" } }]

In [14]:
new_response_code = _[0]
new_response_code

NewResponseCode { endpoint: Endpoint(157.230.134.224:2016), code: LoggedIn { cookie: "Q-xdaM5GMaRq-z7S", server_version: "0.3.5" } }

In [15]:
cookie = new_response_code.response_code.cookie
cookie

'Q-xdaM5GMaRq-z7S'

### Convenience function

In [16]:
async def request_action_to_filter(req_action, endpoint=chococat_endpoint):
    filter_cmd = FilterCmdW("sendrequestaction", endpoint=endpoint, action=req_action)
    return await fiface.command_response(filter_cmd)

### Join a room

In [17]:
await request_action_to_filter(RequestActionW("joinroom", room_name="general"))

LTI136 skipped 51: received transport_cmd SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 0 }, retry_interval: 250ms }], packets: [Request { sequence: 2, response_ack: Some(0), cookie: Some("Q-xdaM5GMaRq-z7S"), action: JoinRoom { room_name: "general" } }] }
LTI137: got transport_rsp back from Transport layer: TransportRsp::Accepted


Accepted

LTI138: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Response { sequence: 0, request_ack: None, code: KeepAlive } }]
LTI139: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Response { sequence: 0, request_ack: None, code: Unauthorized { error_msg: "not a unique name" } } }]
LTI140: got transport notifications: []


Due to a bug in the Netwayste v1 server, there is a `BadRequest { error_msg: "cannot join game because in-game" }` filter notice following the `JoinedRoom` filter notice, below. Apparently, it's caused by a packet retry of the `JoinRoom` request action by the Transport layer, which is erroneously being treated as a second request action by the NWv1 server. It's erroneous because the request sequence is the same, so it should have been dropped by the server.

In [18]:
fiface.get_notifications()

[]

In [19]:
fiface.get_notifications()

[]

In [20]:
fiface.get_notifications()

[]